In [25]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import wrds
import importlib
import volpy_func_lib as vp
import wrds_lib2 as wl

importlib.reload(wl)
importlib.reload(vp)

db = wrds.Connection(wrds_username="axelledue") 

Loading library list...
Done


# create option, forward and return csv files

In [24]:
importlib.reload(wl)

tickers = vp.DJX_tickers
tickers = vp.Market_tickers2  # ['AAPL'] # 'IBM', 'GE', 'UNH', 'ABT','DIS', 'DJX'

wl.fetch_wrds_data_per_ticker(db, 
                              tickers = tickers, #['AAPL','MSFT','TSLA','AMZN','NVDA']
                              chunk_size=1000000,
                              chunk_days=200,
                              data_types=["O", "F", "S", "Z"]) # S_OM hvis OM stock file ["O", "F", "S", "Z"]

Importing option data:   2%|▏         | 3/195 [04:33<4:52:04, 91.28s/ticker, ticker=FDX]


KeyboardInterrupt: 

## close database 

In [ ]:
db.close

In [20]:
import pandas as pd
dftemp = pd.read_excel(r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents with bloomberg data.xlsx", dtype=str)

In [19]:
# Antag at dftemp["ID_CUSIP"] indeholder fulde 9-cifrede CUSIPs
cusips_full = dftemp["ID_CUSIP"].dropna().astype(str)

# Fjern sidste tegn (check digit) → behold kun de første 8
cusips_8 = cusips_full.str[:8]

# Fjern dubletter og lav en komma-separeret streng med citationstegn
cusip_list_string = ", ".join(f"'{c}'" for c in sorted(cusips_8.unique()))


In [22]:
import wrds

query = f"""
select distinct permno, cusip, ticker, comnam
from crsp.dsenames
where substr(cusip, 1, 8) in ({cusip_list_string})
"""

result = db.raw_sql(query)


In [25]:
result.sort_values("permno",ascending=True).head(50)

,permno,cusip,ticker,comnam
455,10104,68389X10,ORCL,ORACLE SYSTEMS CORP
358,10104,68389X10,ORCL,ORACLE CORP
324,10107,59491810,MSFT,MICROSOFT CORP
233,10145,43851610,ALD,ALLIED SIGNAL INC
175,10145,43851610,HON,HONEYWELL INTERNATIONAL INC
433,10145,43851610,<NA>,ALLIED CHEMICAL & DYE CORP
182,10145,43851610,ACD,ALLIED CHEMICAL CORP
304,10145,43851610,ALD,ALLIED CORP
87,10145,43851610,<NA>,ALLIED CHEMICAL CORP
250,10147,26864810,EMC,E M C CORP MA


In [ ]:
import pandas as pd
import wrds

# 1) Antag du allerede har cusips_8 som en pandas Series
cusips_full = dftemp["ID_CUSIP"].dropna().astype(str)
cusips_8 = cusips_full.str[:8].drop_duplicates().reset_index(drop=True)

# Lav en DataFrame alene med dine CUSIPs
df_cusip = pd.DataFrame({"cusip8": cusips_8})

# 2) Hent matches fra WRDS
query = f"""
    select distinct permno, substr(cusip,1,8) as cusip8
    from crsp.dsenames
    where substr(cusip, 1, 8) in ({cusip_list_string})
"""
matches = db.raw_sql(query)

# 3) Merge for at finde ikke-matches
merged = df_cusip.merge(matches, on="cusip8", how="left")

# 4) De rækker hvor permno er NaN, har ingen match i CRSP
no_match = merged[merged["permno"].isna()]

print("Cusips uden match (første 10):")
print(no_match.head(10))
print(f"I alt {len(no_match)} ud af {len(df_cusip)} cusips manglede match.")


In [39]:
import pandas as pd
import wrds

# Forbind til WRDS

# 1. Ekstrahér de første 8 tegn af CUSIPs
cusips_full = dftemp["ID_CUSIP"].dropna().astype(str)
cusips_8 = cusips_full.str[:8].drop_duplicates()
cusip_list_string = ", ".join(f"'{c}'" for c in sorted(cusips_8))

# 2. SQL-spørgsmål til CRSP
query = f"""
    select distinct substr(cusip, 1, 8) as cusip, permno, ticker
    from crsp.dsenames
    where substr(cusip, 1, 8) in ({cusip_list_string})
"""
matches = db.raw_sql(query)

# 3. Merge for at inkludere alle cusips – også uden match
df_cusip = pd.DataFrame({"cusip": cusips_8})
result = df_cusip.merge(matches, on="cusip", how="left")

# 4. Gem til CSV
result.to_csv(r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents_map.csv", index=False)
print("CSV gemt: cusip_permno_ticker_matches.csv")


CSV gemt: cusip_permno_ticker_matches.csv


In [22]:
import pandas as pd
import wrds

# 1. Ekstrahér de første 8 tegn af CUSIPs
cusips_full = dftemp["ID_CUSIP"].dropna().astype(str)
cusips_8 = cusips_full.str[:8].drop_duplicates()
cusip_list_string = ", ".join(f"'{c}'" for c in sorted(cusips_8))

# 2. Find PERMNO baseret på CUSIP
query_permno = f"""
    select distinct substr(cusip, 1, 8) as cusip, permno
    from crsp.dsenames
    where substr(cusip, 1, 8) in ({cusip_list_string})
"""
cusip_to_permno = db.raw_sql(query_permno)

# 3. Find TICKER baseret på PERMNO
permnos_unique = cusip_to_permno["permno"].dropna().unique()
permno_list_string = ", ".join(str(p) for p in sorted(permnos_unique))

query_ticker = f"""
    select permno, ticker
    from crsp.dsenames
    where permno in ({permno_list_string})
"""
permno_to_ticker = db.raw_sql(query_ticker)

# 4. Behold én ticker pr PERMNO
permno_to_ticker = permno_to_ticker.drop_duplicates(subset="permno", keep="last")

# 5. Merge resultater
df_cusip = pd.DataFrame({"cusip": cusips_8})
result = df_cusip.merge(cusip_to_permno, on="cusip", how="left").merge(permno_to_ticker, on="permno", how="left")

# 6. Konverter alt til string
result = result.astype(str)

# 7. Gem som Excel-fil
output_path = r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents_map.xlsx"
result.to_excel(output_path, index=False)
print(f"✔️ Excel gemt: {output_path}")


✔️ Excel gemt: C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents_map.xlsx


In [19]:
import pandas as pd
import wrds

# 1. Forbind til WRDS
# 2. Indlæs Excel-fil med CUSIP, PERMNO, Ticker
path = r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents_map_crsp_rdy.xlsx"
df_map = pd.read_excel(path, dtype=str)  # Tving alt til at være string

# 3. Klargør CUSIP'er
cusips_8 = df_map["cusip"].dropna().str[:8].drop_duplicates()
cusip_list_string = ", ".join(f"'{c}'" for c in cusips_8)

# 4. Træk alle secid'er fra optionm.securd1 baseret på CUSIP
query = f"""
    SELECT DISTINCT cusip, secid
    FROM optionm.securd1
    WHERE substr(cusip, 1, 8) IN ({cusip_list_string})
"""
cusip_to_secid = db.raw_sql(query)

# 5. Klargør til merge – harmonisér CUSIP til 8 tegn
df_map["cusip"] = df_map["cusip"].str[:8]
cusip_to_secid["cusip"] = cusip_to_secid["cusip"].str[:8]

# 6. Merge for at tilføje SECID'er til det originale map
result = df_map.merge(cusip_to_secid, on="cusip", how="left")

# 7. Gem som Excel
output_path = r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\cusip_secid_map.xlsx"
result.to_excel(output_path, index=False)
print(f"✔️ Fil gemt som Excel: {output_path}")


✔️ Fil gemt som Excel: C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\cusip_secid_map.xlsx


In [74]:
import pandas as pd

# 1. Load DJX-data
djx_df = pd.read_excel(
    r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents with bloomberg data.xlsx"
)
djx_df = djx_df[djx_df['kilde'] == 'INDU']

# 2. Load det brede ark
constituents_df = pd.read_excel(
    r"C:\Users\axell\Desktop\CBS\data\constituents2.xlsx", header=[0, 1]
)

# 3. Smelt det brede ark til langt format
long_df = pd.DataFrame()
for date in constituents_df.columns.levels[0]:
    if 'ID' in constituents_df[date].columns:
        tmp = constituents_df[[date]].copy()
        tmp.columns = tmp.columns.droplevel(0)
        tmp['date'] = pd.to_datetime(date, format="%d-%m-%Y")
        tmp['Ticker'] = tmp['ID']
        long_df = pd.concat([long_df, tmp[['date', 'Ticker']]])

# 4. Rens og merge
long_df.dropna(subset=['Ticker'], inplace=True)
long_df.drop_duplicates(inplace=True)

# Merge
merged_df = pd.merge(
    long_df,
    djx_df[['Ticker', 'LONG_COMP_name', 'ID_CUSIP']],
    on='Ticker',
    how='left'
)

# 5. Trim CUSIP til 8 karakterer og behold som tekst
merged_df['ID_CUSIP'] = merged_df['ID_CUSIP'].astype(str).str[:8]

# 6. Gem til Excel uden ekstra pakker
merged_df.to_excel(r"C:\Users\axell\Desktop\CBS\data\djx_constituents_long2.xlsx", index=False)


In [70]:
import pandas as pd

# 1. Load OEX-data
oex_df = pd.read_excel(
    r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\constituents with bloomberg data.xlsx"
)
oex_df = oex_df[oex_df['kilde'] == 'OEX'].copy()

# 2. Load constituents-arket
constituents_df = pd.read_excel(
    r"C:\Users\axell\Desktop\CBS\data\constituents.xlsx", header=[0, 1], sheet_name="OEXQ"
)
# 3. Smelt det brede ark til langt format
long_df = pd.DataFrame()
for date in constituents_df.columns.levels[0]:
    if 'ID' in constituents_df[date].columns:
        tmp = constituents_df[[date]].copy()
        tmp.columns = tmp.columns.droplevel(0)
        tmp['date'] = pd.to_datetime(date, format="%d-%m-%Y")
        tmp['Ticker'] = tmp['ID']
        long_df = pd.concat([long_df, tmp[['date', 'Ticker']]])

# 4. Rens og merge
long_df.dropna(subset=['Ticker'], inplace=True)
long_df.drop_duplicates(inplace=True)

# Merge
merged_df = pd.merge(
    long_df,
    oex_df[['Ticker', 'LONG_COMP_name', 'ID_CUSIP']],
    on='Ticker',
    how='left'
)

# 5. Trim CUSIP til 8 karakterer og behold som tekst
merged_df['ID_CUSIP'] = merged_df['ID_CUSIP'].astype(str).str[:8]

# 6. Gem til Excel uden ekstra pakker
merged_df.to_excel(r"C:\Users\axell\Desktop\CBS\data\oex_constituents_long2.xlsx", index=False)


In [61]:
dftemp2 = pd.read_excel(r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\final map done.xlsx")

# 1. For CUSIP – tjek om én cusip maps til flere PERMNO, SECID, TICKER
print("CUSIP til flere værdier:")
print(dftemp2.groupby("cusip").agg({
    "permno": pd.Series.nunique,
    "secid": pd.Series.nunique,
    "ticker": pd.Series.nunique
}).query("permno > 1 or secid > 1 or ticker > 1"))

# 2. For PERMNO – tjek om én permno maps til flere CUSIP, SECID, TICKER
print("\nPERMNO til flere værdier:")
print(dftemp2.groupby("permno").agg({
    "cusip": pd.Series.nunique,
    "secid": pd.Series.nunique,
    "ticker": pd.Series.nunique
}).query("cusip > 1 or secid > 1 or ticker > 1"))

# 3. For SECID – tjek om én secid maps til flere CUSIP, PERMNO, TICKER
print("\nSECID til flere værdier:")
print(dftemp2.groupby("secid").agg({
    "cusip": pd.Series.nunique,
    "permno": pd.Series.nunique,
    "ticker": pd.Series.nunique
}).query("cusip > 1 or permno > 1 or ticker > 1"))

# 4. For TICKER – tjek om én ticker maps til flere CUSIP, PERMNO, SECID
print("\nTICKER til flere værdier:")
print(dftemp2.groupby("ticker").agg({
    "cusip": pd.Series.nunique,
    "permno": pd.Series.nunique,
    "secid": pd.Series.nunique
}).query("cusip > 1 or permno > 1 or secid > 1"))


CUSIP til flere værdier:
Empty DataFrame
Columns: [permno, secid, ticker]
Index: []

PERMNO til flere værdier:
Empty DataFrame
Columns: [cusip, secid, ticker]
Index: []

SECID til flere værdier:
Empty DataFrame
Columns: [cusip, permno, ticker]
Index: []

TICKER til flere værdier:
Empty DataFrame
Columns: [cusip, permno, secid]
Index: []


In [79]:
import pandas as pd

# 1. Load mapping-fil (alle kolonner som tekst)
map_path = r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\final map done_v2.xlsx"
df_map = pd.read_excel(map_path, dtype=str)

# 2. Load DJX constituents
djx_path = r"C:\Users\axell\Desktop\CBS\data\djx_constituents_long2.xlsx"
df_djx = pd.read_excel(djx_path, dtype=str)

# 3. Trim CUSIP til 8 karakterer i begge datasæt (sikkerhed)
df_map["cusip"] = df_map["cusip"].str.strip().str[:8]
df_djx["ID_CUSIP"] = df_djx["ID_CUSIP"].str.strip().str[:8]

# 4. Merge på CUSIP
merged_djx_mapped = pd.merge(
    df_djx,
    df_map[["cusip", "permno", "secid", "ticker"]],
    left_on="ID_CUSIP",
    right_on="cusip",
    how="left"
)

# Fjern rækker hvor alle match-kolonner er NaN (dvs. intet blev fundet i mapping)
merged_djx_cleaned = merged_djx_mapped.dropna(subset=["permno", "secid", "ticker"], how="all")

# 6. Konverter alt til str for at undgå Excel-formatproblemer
merged_djx_cleaned = merged_djx_cleaned.astype(str)

# 7. Gem som Excel v3
output_path = r"C:\Users\axell\Desktop\CBS\data\djx_constituents_long3.xlsx"
merged_djx_cleaned.to_excel(output_path, index=False)

print(f"✔️ Gemt som: {output_path}")


✔️ Gemt som: C:\Users\axell\Desktop\CBS\data\djx_constituents_long3.xlsx


In [80]:
import pandas as pd

# 1. Load mapping-fil (alle kolonner som tekst)
map_path = r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\final map done_v2.xlsx"
df_map = pd.read_excel(map_path, dtype=str)

# 2. Load OEX constituents
oex_path = r"C:\Users\axell\Desktop\CBS\data\oex_constituents_long2.xlsx"
df_oex = pd.read_excel(oex_path, dtype=str)

# 3. Trim CUSIP til 8 karakterer i begge datasæt (sikkerhed)
df_map["cusip"] = df_map["cusip"].str.strip().str[:8]
df_oex["ID_CUSIP"] = df_oex["ID_CUSIP"].str.strip().str[:8]

# 4. Merge på CUSIP
merged_oex_mapped = pd.merge(
    df_oex,
    df_map[["cusip", "permno", "secid", "ticker"]],
    left_on="ID_CUSIP",
    right_on="cusip",
    how="left"
)

# 5. Fjern rækker hvor alle match-kolonner er NaN (dvs. intet blev fundet i mapping)
merged_oex_cleaned = merged_oex_mapped.dropna(subset=["permno", "secid", "ticker"], how="all")

# 6. Konverter alt til str for at undgå Excel-formatproblemer
merged_oex_cleaned = merged_oex_cleaned.astype(str)

# 7. Gem som Excel v3
output_path = r"C:\Users\axell\Desktop\CBS\data\oex_constituents_long3.xlsx"
merged_oex_cleaned.to_excel(output_path, index=False)

print(f"✔️ Gemt som: {output_path}")


✔️ Gemt som: C:\Users\axell\Desktop\CBS\data\oex_constituents_long3.xlsx


In [14]:
import pandas as pd

dftemp = pd.read_excel(r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\oex_constituents_long3.xlsx")

In [15]:
tickers = dftemp["ticker"].dropna().unique().tolist()
ticker_list = [f'"{t}"' for t in tickers]
print(f"[{', '.join(ticker_list)}]")


["NSM", "G", "DOW_chem", "DD_eidp", "LU", "MEDI", "EMC", "CCU", "UBAN", "HCA", "ONE", "S_sears", "T_old", "PHA", "AGC", "MAY", "AA", "AEP", "AES", "AIG", "AMGN", "ATI", "AVP", "AXP", "BA", "BAC", "BAX", "BBWI", "BDK", "BHGE", "BMY", "BNI", "C", "CGP", "CI", "CL", "CPB", "DXC", "CSCO", "DAL", "DIS", "EK", "ENE", "ETR", "EXC", "F", "FDX", "GX", "GD", "GE", "HAL", "HD", "HET", "HIG", "HNZ", "HON", "HPQ", "HSH", "IBM", "INTC", "IP", "JCI", "JNJ", "JPM", "KO", "LEH", "MCD", "MER", "MMM", "MRK", "MS", "MSFT", "NT", "NSC", "NXTL", "OMX", "ORCL", "PARA", "PEP", "PFE", "PG", "RAL", "ROK", "RSH", "RTN", "RTX", "SLB", "SO", "TOY", "TWX", "TXN", "UIS", "VZ", "WFC", "WMB", "WMT", "WY", "XOM", "XRX", "EP", "USB", "T", "BUD", "MDT", "MO", "GS", "ALL", "DELL", "CMCSA", "S", "ABT", "CAT", "TGT", "CVX", "UPS", "WB", "COF", "COP", "GOOGL", "RF", "CVS", "AAPL", "MDLZ", "BK", "COV", "NYX", "PM", "UNH", "NOV", "MA", "OXY", "QCOM", "DVN", "GILD", "LMT", "LOW", "NKE", "SGP", "WBA", "WYE", "MON", "AMZN", "COST

In [ ]:
vp.Market_tickers2

vp.VIX_tickers

['UK',
 'DD_eidp',
 'JPM',
 'S_sears',
 'AA',
 'AXP',
 'BA',
 'C',
 'CAT',
 'CVX',
 'DIS',
 'EK',
 'GE',
 'GT',
 'HON',
 'HPQ',
 'IBM',
 'IP',
 'JNJ',
 'KO',
 'MCD',
 'MMM',
 'MO',
 'MRK',
 'PG',
 'RTX',
 'T',
 'WMT',
 'XOM',
 'T_old',
 'HD',
 'INTC',
 'MSFT',
 'AIG',
 'PFE',
 'VZ',
 'BAC',
 'MDLZ',
 'CSCO',
 'TRV',
 'UNH',
 'GS',
 'NKE',
 'V',
 'AAPL',
 'DD',
 'WBA',
 'DOW_inc',
 'AMGN',
 'CRM',
 'AMZN',
 'NVDA',
 'SHW',
 'NSM',
 'G',
 'DOW_chem',
 'DD_eidp',
 'LU',
 'MEDI',
 'EMC',
 'CCU',
 'UBAN',
 'HCA',
 'ONE',
 'S_sears',
 'T_old',
 'PHA',
 'AGC',
 'MAY',
 'AA',
 'AEP',
 'AES',
 'AIG',
 'AMGN',
 'ATI',
 'AVP',
 'AXP',
 'BA',
 'BAC',
 'BAX',
 'BBWI',
 'BDK',
 'BHGE',
 'BMY',
 'BNI',
 'C',
 'CGP',
 'CI',
 'CL',
 'CPB',
 'DXC',
 'CSCO',
 'DAL',
 'DIS',
 'EK',
 'ENE',
 'ETR',
 'EXC',
 'F',
 'FDX',
 'GX',
 'GD',
 'GE',
 'HAL',
 'HD',
 'HET',
 'HIG',
 'HNZ',
 'HON',
 'HPQ',
 'HSH',
 'IBM',
 'INTC',
 'IP',
 'JCI',
 'JNJ',
 'JPM',
 'KO',
 'LEH',
 'MCD',
 'MER',
 'MMM',
 'MRK',
 'MS',
 'M

In [18]:
missing = set(vp.VIX_tickers) - set(vp.Market_tickers2)
print(sorted(missing))


['DIA', 'EEM', 'EWZ', 'FXE', 'IWM', 'QQQ', 'SLV', 'SPX', 'USO']


In [19]:
vp.Market_tickers2

['UK',
 'DD_eidp',
 'JPM',
 'S_sears',
 'AA',
 'AXP',
 'BA',
 'C',
 'CAT',
 'CVX',
 'DIS',
 'EK',
 'GE',
 'GT',
 'HON',
 'HPQ',
 'IBM',
 'IP',
 'JNJ',
 'KO',
 'MCD',
 'MMM',
 'MO',
 'MRK',
 'PG',
 'RTX',
 'T',
 'WMT',
 'XOM',
 'T_old',
 'HD',
 'INTC',
 'MSFT',
 'AIG',
 'PFE',
 'VZ',
 'BAC',
 'MDLZ',
 'CSCO',
 'TRV',
 'UNH',
 'GS',
 'NKE',
 'V',
 'AAPL',
 'DD',
 'WBA',
 'DOW_inc',
 'AMGN',
 'CRM',
 'AMZN',
 'NVDA',
 'SHW',
 'NSM',
 'G',
 'DOW_chem',
 'DD_eidp',
 'LU',
 'MEDI',
 'EMC',
 'CCU',
 'UBAN',
 'HCA',
 'ONE',
 'S_sears',
 'T_old',
 'PHA',
 'AGC',
 'MAY',
 'AA',
 'AEP',
 'AES',
 'AIG',
 'AMGN',
 'ATI',
 'AVP',
 'AXP',
 'BA',
 'BAC',
 'BAX',
 'BBWI',
 'BDK',
 'BHGE',
 'BMY',
 'BNI',
 'C',
 'CGP',
 'CI',
 'CL',
 'CPB',
 'DXC',
 'CSCO',
 'DAL',
 'DIS',
 'EK',
 'ENE',
 'ETR',
 'EXC',
 'F',
 'FDX',
 'GX',
 'GD',
 'GE',
 'HAL',
 'HD',
 'HET',
 'HIG',
 'HNZ',
 'HON',
 'HPQ',
 'HSH',
 'IBM',
 'INTC',
 'IP',
 'JCI',
 'JNJ',
 'JPM',
 'KO',
 'LEH',
 'MCD',
 'MER',
 'MMM',
 'MRK',
 'MS',
 'M

In [20]:
dftemp = pd.read_excel(r"C:\Users\axell\Desktop\CBS\data\OptionMetrics\Tickers\index data\final map done_v2.xlsx")